[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DSIMB/PoincareMSA/blob/master/PoincareMSA_colab.ipynb)

<img src="https://github.com/DSIMB/PoincareMSA/blob/master/.github/PoincareMSA_small_logo.png?raw=true" height="100" style="height:100px;margin-left: 0px;">

# Poincaré maps for visualization of large protein famillies

**Authors**: Anna Klimovskaia Susmelj, Yani Ren, Yann Vander Meersche, Jean-Christophe Gelly and Tatiana Galochkina

PoincaréMSA builds an interactive projection of an input protein multiple sequence alignemnt (MSA) using a method based on Poincaré maps described by Klimovskaia et al [1]. It reproduces both local proximities of protein sequences and hierarchy contained in give data. Thus, sequences located closer to the center of projection correspond to the proteins sharing the most general functional properites and/or appearing at the earlier stages of evolution. Source code is available at https://github.com/DSIMB/PoincareMSA.

[1] Klimovskaia, A., Lopez-Paz, D., Bottou, L. et al. Poincaré maps for analyzing complex hierarchies in single-cell data. Nat Commun 11, 2966 (2020).

# Notebook initialization

In [ ]:
# Update working directory

# %cd /home/hugo/Bureau/PoincareMSA/

/home/hugo/Bureau/PoincareMSA


In [7]:
# import sys
# sys.path.append('/home/hugo/Bureau/PoincareMSA')

In [8]:
#Load dependencies
import os
import numpy as np
import pandas as pd
import subprocess
import json
import warnings
warnings.filterwarnings('ignore')

#Import visualization functions
from scripts.visualize_projection.pplots_new import read_embeddings, plot_embedding, plot_embedding_interactive, rotate, get_colors
from scripts.prepare_data.mmseqs2_api import run_mmseqs2
from scripts.prepare_data.uniprot_idmapping_api import submit_id_mapping, check_id_mapping_results_ready, get_id_mapping_results_link, get_id_mapping_results_search
%matplotlib inline

#Create optional variables
path_annotation = ""

# Data import

In [9]:
# OPTIONS =================================================
#MSA in mfasta format OR embedding path (the other must be none)
mfasta = 'examples/kinases/kinases.mfasta'
embedding_path = 'embeddings/ankh_base_kinases/'

#Annotation file (.csv) or UniProt ID list. (Emtpy strings for no annotations)
path_annotation_csv = "examples/kinases/kinase_group_new.csv"   # Path or ""
# OR
path_uniprot_list = ""   # Path or ""
#==========================================================

In [10]:
#Check files
#mfasta
nb_seq = 0
if mfasta is not None:
    embed = False
    if os.path.isfile(mfasta):
        with open(mfasta, "r") as f_in:
            for line in f_in:
                if line[0] == ">":
                    nb_seq += 1
        print(f"\nNumber of sequences found: {nb_seq}.")
    else:
        print(f"File {mfasta} not found.")

if embedding_path is not None:
    embed = True
    if os.path.exists(embedding_path):
        nb_seq = len([s for s in os.listdir(embedding_path) if '.pt' in s])
        print(f"\nNumber of sequences found: {nb_seq}.")
    else:
        print(f"Folder {embedding_path} not found.")
else:
    print('Neither a valid mfasta or embedding folder has been provided')

#Check that only one path is selected
if path_annotation_csv and path_uniprot_list:
    raise ValueError("Use only one file path (path_annotation_csv OR path_uniprot_list).")

#Check that only one path is selected
if path_annotation_csv and path_uniprot_list:
    raise ValueError("Use only one file path (path_annotation_csv OR path_uniprot_list).")

if path_annotation_csv:
    if os.path.isfile(path_annotation_csv):
        try:
            df_annotation = pd.read_csv(path_annotation_csv)
        except:
            raise ValueError("Annotation file is not in .csv format.")
        else:
            if len(df_annotation) != nb_seq:
                raise ValueError("Annotation file doesn't match the .mfasta file length.")

        #Add id column
        if "proteins_id" not in df_annotation.columns:
            df_annotation.insert(0, "proteins_id", range(len(df_annotation)))
        path_annotation = path_annotation_csv
            
        print("\nAnnotation file correctly loaded.")
        annotation_names = list(df_annotation.columns)
        print(f"{len(annotation_names)} annotations found: {annotation_names}.")
    else:
        print(f"File {path_annotation_csv} not found.")

if path_uniprot_list:
    if os.path.isfile(path_uniprot_list):
        try:
            UnP_ids = np.genfromtxt(path_uniprot_list, dtype="str")
        except:
            raise ValueError("UniProt IDs file is not in a valid format.")
        else:
            if len(UnP_ids) != nb_seq:
                raise ValueError("UniProt IDs file doesn't match the .mfasta file length.")     

            #Split UniProtKB and UniParc IDs
            uniparc_ids = []
            uniprot_ids = []
            for unp in UnP_ids:
                if len(unp) == 13 and unp[:2] == "UP":
                    uniparc_ids.append(unp)
                else:
                    uniprot_ids.append(unp)

            #Fetch UniProtKB annotations
            job_id = submit_id_mapping(
                from_db="UniProtKB_AC-ID", to_db="UniParc", ids=uniprot_ids
            )

            if check_id_mapping_results_ready(job_id):
                link = get_id_mapping_results_link(job_id)
                results = get_id_mapping_results_search(link)

            #Fetch UniParc annotations
            job_id = submit_id_mapping(
                from_db="UniParc", to_db="UniParc", ids=uniparc_ids
            )

            if check_id_mapping_results_ready(job_id):
                link = get_id_mapping_results_link(job_id)
                results2 = get_id_mapping_results_search(link)

            #Create annotation dataframe
            df_annotation = pd.DataFrame(UnP_ids, columns=["UnP_ID"])
            df_annotation[["organism", "proteinName", "taxonId", "species", "genus", \
                           "family", "order", "class", "phylum", "clade", "superkingdom"]] = ""

            #Fill the annotation DataFrame
            for dict_res in results["results"] + results2["results"]:
                try:
                    unp = dict_res["from"]
                    prot_name = dict_res["to"]["uniParcCrossReferences"][0]["proteinName"]
                    df_annotation.loc[df_annotation["UnP_ID"] == unp, "proteinName"] = prot_name
                    scientific_name = dict_res["to"]["uniParcCrossReferences"][0]["organism"]["scientificName"]
                    taxid = dict_res["to"]["uniParcCrossReferences"][0]["organism"]["taxonId"]
                    df_annotation.loc[df_annotation["UnP_ID"] == unp, "organism"] = scientific_name
                    df_annotation.loc[df_annotation["UnP_ID"] == unp, "taxonId"] = taxid
                except KeyError:
                    continue

            #Add lineage from NCBI Taxonomist
            taxon_ids = df_annotation.loc[df_annotation["taxonId"].notnull(), 'taxonId'].to_numpy()
            taxon_ids = list(set(taxon_ids))
            taxon_ids = list(map(str, taxon_ids))
            bash_command = f"ncbi-taxonomist resolve -t {','.join(taxon_ids)}"
            list_taxon = subprocess.run(bash_command, shell=True, capture_output=True, text=True).stdout.strip().split("\n")

            if list_taxon != [""]:
                for taxon in list_taxon:
                    jsonString = taxon
                    taxon_dict = json.loads(jsonString)
                    query = taxon_dict["query"]
                    for lineage in taxon_dict["lineage"]:
                        rank = lineage["rank"]
                        if rank in ["species", "genus", "family", "order", "class", "phylum", "clade", "superkingdom"]:
                            name = lineage["name"]
                            df_annotation.loc[df_annotation["taxonId"] == int(query), rank] = name

            #Add id column
            if "proteins_id" not in df_annotation.columns:
                df_annotation.insert(0, "proteins_id", range(len(df_annotation)))

            #Save annotation to csv
            path_annotation = "auto_annot.csv"
            df_annotation.to_csv(path_annotation, index=False)

            print("\nAnnotation correctly fetched.")
            annotation_names = list(df_annotation.columns)
            print(f"{len(annotation_names)} annotations found: {annotation_names}.")
    else:
        print(f"File {path_uniprot_list} not found.")


Number of sequences found: 497.

Number of sequences found: 497.

Annotation file correctly loaded.
19 annotations found: ['proteins_id', '1_Group', '2_Gene', '3_HGNC', '4_Uni_entry', '5_Uni_acc', '6_Domain_begin', '7_Domain_end', '8_Domain_length', '9_Largest_insert_length', '10_PDB_validation', '11_Conformational_state', '12_Dihedral_state', '13_Group_in_Uni', '14_Group_in_Manning', '15_Synonymn', 'evo_distance', 'decile_domain', 'small_cluster'].


# Data preparation
Here we clean the input .mfasta alignment and translate each sequence to a vector ready for projection.

### Parameters for data preparation

In [11]:
# OPTIONS =================================================
# Job name
#Name for the output folder
out_name_mfasta = "kinases_data"
#----------------------------------------------------------
# Threshold for filtering gapped positions
#Positions with proportion of gaps above the given threshold are removed from the alignment.
#If your alignment is very gapped, you may want to increase this value.
gapth_mfasta = 0.9 
#==========================================================


# Run data preparation
#Data preparation consists in `.mfasta` cleaning according to a gap threshold and
#translation of each sequence to the PSSM profile.

prep_parameters_mfasta = "scripts/prepare_data" + " " + mfasta + " " + out_name_mfasta + " " + out_name_mfasta + " " + str(gapth_mfasta)
bash_projection = "bash scripts/prepare_data/create_projection.sh " + prep_parameters_mfasta
!{bash_projection}

Input file: examples/kinases/kinases.mfasta
Name of the protein family: kinases
23923 X aa replaced by gaps in 497 sequences
filter_gaps finished for examples/kinases/kinases.mfasta
mfasta2fasta finished for kinases_data/kinases_data.clean0.9.mfasta
23923 X aa replaced by gaps in 497 sequences


# Projection

### Projection parameters (no embeddings)

In [12]:
### Projection parameters (no embeddings)
# EMBEDDING SELECTION
in_name_mfasta = 'examples/kinases/kinases_data/fasta0.9/'  # Input here the name of folder with the fasta files
mid_output_mfasta = 'kinases_data/with_mfasta/'  # Input name of desired folder for intermediary results
out_name_mfasta = 'results/kinases/with_mfasta/' # Input desired name of output folder

# OPTIONS =================================================
#Here you control different parameters of Poincaré maps.
#In our computational experiments the best results were achieved for the following values provided by default.
#The impact of different parameters is analyzed in the original paper [1].
knn_mfasta = 5
gamma_mfasta = 2
sigma_mfasta = 1
cospca_mfasta = 0
batchs_mfasta = 4
epochs_mfasta = 1000
seed_mfasta = 4
#==========================================================


# Building projection and preparing data for visualization
#This step creates a projection of encoded sequences to a Poincaré disk.
# bash_pm = "python3 "+ "scripts/build_poincare_map/main.py --input_path " + out_name + "/fasta" + str(gapth) + " --output_path " + out_name + "/projections/ --gamma "+ str(gamma) +" --pca "+ str(cospca) + " --epochs "+ str(epochs) +" --seed "+ str(seed) + " --knn " + str(knn)
# !{bash_pm}
bash_pm_mfasta = "python "+ "scripts/build_poincare_map/main.py --input_path " + in_name_mfasta + \
          " --output_path " + out_name_mfasta + " --plm_embedding False" +  " --matrices_output_path " + mid_output_mfasta \
          + " --gamma "+ str(gamma_mfasta) +" --pca "+ str(cospca_mfasta) + " --epochs "+ str(epochs_mfasta) +" --seed "\
          + str(seed_mfasta) + " --knn " + str(knn_mfasta)
!{bash_pm_mfasta}

CUDA: True
Random seed set as 4
497 proteins found in folder examples/kinases/kinases_data/fasta0.9/.
No root detected
['74.aamtx', '358.aamtx', '433.aamtx', '445.aamtx', '128.aamtx', '246.aamtx', '57.aamtx', '421.aamtx', '62.aamtx', '61.aamtx', '320.aamtx', '136.aamtx', '337.aamtx', '223.aamtx', '200.aamtx', '446.aamtx', '50.aamtx', '190.aamtx', '217.aamtx', '289.aamtx']
74.aamtx
Prepare data: tensor construction
Prepare data: successfully terminated
labels: ['74' '358' '433' '445' '128' '246' '57' '421' '62' '61' '320' '136' '337'
 '223' '200' '446' '50' '190' '217' '289' '378' '13' '189' '372' '366'
 '351' '135' '348' '241' '48' '302' '400' '341' '202' '89' '242' '49'
 '347' '203' '144' '97' '424' '450' '20' '90' '380' '172' '105' '207'
 '304' '461' '419' '408' '279' '316' '160' '17' '486' '99' '139' '4' '371'
 '282' '326' '284' '181' '431' '467' '164' '85' '409' '171' '267' '6'
 '480' '197' '458' '278' '67' '251' '233' '98' '100' '77' '280' '393'
 '209' '401' '277' '402' '275' '459

### Projection parameters (with embeddings)

In [13]:
### Projection parameters (with embeddings)


# EMBEDDING SELECTION
in_name_embed = 'embeddings/ankh_base_kinases/'  # Input here the name of folder with the embeddings
mid_output_embed = 'kinases_data/with_plm_embeddings/'  # Input name of desired folder for intermediary results
out_name_embed = 'results/kinases/with_plm_embeddings/' # Input desired name of output folder


# OPTIONS =================================================
#Here you control different parameters of Poincaré maps.
#In our computational experiments the best results were achieved for the following values provided by default.
#The impact of different parameters is analyzed in the original paper [1].
knn_embed = 5
gamma_embed = 2
sigma_embed = 1
cospca_embed = 0
batchs_embed = 4
epochs_embed = 1000
seed_embed = 0
#==========================================================


# Building projection and preparing data for visualization
#This step creates a projection of encoded sequences to a Poincaré disk.

bash_pm_embed = "python "+ "scripts/build_poincare_map/main.py --input_path " + in_name_embed + \
                " --output_path " + out_name_embed + " --plm_embedding True" +  " --matrices_output_path " + mid_output_embed +\
                " --distlocal minkowski" + " --gamma "+ str(gamma_embed) +" --pca "+ str(cospca_embed) + " --epochs "+ str(epochs_embed) +" --seed "\
                + str(seed_embed) + " --knn " + str(knn_embed)
!{bash_pm_embed}

CUDA: True
Random seed set as 0
497 proteins found in folder embeddings/ankh_base_kinases/.
['416.pt', '379.pt', '458.pt', '230.pt', '53.pt', '450.pt', '367.pt', '6.pt', '150.pt', '451.pt', '259.pt', '381.pt', '236.pt', '188.pt', '235.pt', '21.pt', '117.pt', '71.pt', '193.pt', '52.pt', '399.pt', '371.pt', '95.pt', '3.pt', '377.pt', '330.pt', '443.pt', '327.pt', '444.pt', '284.pt', '391.pt', '345.pt', '103.pt', '492.pt', '258.pt', '376.pt', '239.pt', '240.pt', '438.pt', '104.pt', '175.pt', '159.pt', '309.pt', '375.pt', '158.pt', '291.pt', '141.pt', '253.pt', '155.pt', '197.pt', '163.pt', '456.pt', '8.pt', '245.pt', '67.pt', '37.pt', '26.pt', '142.pt', '14.pt', '94.pt', '397.pt', '78.pt', '125.pt', '487.pt', '56.pt', '152.pt', '160.pt', '490.pt', '341.pt', '238.pt', '136.pt', '38.pt', '172.pt', '66.pt', '378.pt', '131.pt', '156.pt', '495.pt', '329.pt', '217.pt', '243.pt', '138.pt', '180.pt', '170.pt', '178.pt', '288.pt', '22.pt', '140.pt', '478.pt', '203.pt', '250.pt', '428.pt', '441.pt'

# Projection visualization

### Prepare data for visualization

In [14]:
# Prepare data for visualization
#Check that an annotation file was provided. Create a dummy one instead
if not path_annotation:
    df_annotation = pd.DataFrame(list(zip(list(range(1,nb_seq+1)), np.full(nb_seq, "-", dtype=object))), columns=["id", "default"])
    df_annotation.to_csv("dummy_annotation.csv", index=False)
    path_annotation = "dummy_annotation.csv"
    annotation_names = ["proteins_id"]

path_embedding_embed = f"{out_name_embed}/PM{knn_embed:1.0f}sigma={sigma_embed:2.2f}gamma={gamma_embed:2.2f}minkowskipca={cospca_embed:1.0f}_seed{seed_embed:1.0f}.csv"
path_embedding_mfasta = f"{out_name_mfasta}/PM{knn_mfasta:1.0f}sigma={sigma_mfasta:2.2f}gamma={gamma_mfasta:2.2f}cosinepca={cospca_mfasta:1.0f}_seed{seed_mfasta:1.0f}.csv"   

df_embedding_embed = read_embeddings(path_embedding_embed, path_annotation, withroot=False)
df_embedding_mfasta = read_embeddings(path_embedding_mfasta, path_annotation, withroot=False)

#Here are different labels found in your annotation file (if one uploaded):
print(f"{len(annotation_names)} annotations found: {annotation_names}.")

19 annotations found: ['proteins_id', '1_Group', '2_Gene', '3_HGNC', '4_Uni_entry', '5_Uni_acc', '6_Domain_begin', '7_Domain_end', '8_Domain_length', '9_Largest_insert_length', '10_PDB_validation', '11_Conformational_state', '12_Dihedral_state', '13_Group_in_Uni', '14_Group_in_Manning', '15_Synonymn', 'evo_distance', 'decile_domain', 'small_cluster'].


### Create interactive plot

In [15]:
# Construction of custom color palette 
kinase_palette = {-1 : "#c7c7c7", "OTHER": "#c7c7c7", "None" :"#c7c7c7", "NA" : "#c7c7c7", "Uncharacterized" : "#c7c7c7", "root": "#000000",
                  "TYR": "#bd065f", "CMGC": "#d5c203", "TKL": "#997e73","STE": "#80b412", # kinase groups 
                  "CK1": "#0dbae9", "AGC": "#00bba1", "CAMK":  "#1f6ed4", "NEK": "#8ce4fa", "RGC":"#f59a62"}

In [16]:
# OPTIONS =================================================
#Here you can set different parameters to color & annotate the resulting projection:

title = "Poincaré projection of the kinase 1 protein family without embeddings"  

#----------------------------------------------------------
# Select the coloring from annotation .csv file:
labels_name = "1_Group"
# Select classes to label among the "labels_name" or "second_labels_name" column (comma separated list):
second_labels_name = ""
labels_text = []
show_text = False
#----------------------------------------------------------
# Use a custom color palette:
color_palette = kinase_palette #Default: None
use_custom_palette = True
#==========================================================


#Check projection visualization parameters
#Labels name
if labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")
#Second labels name
if second_labels_name == "":
    second_labels_name = None
elif second_labels_name not in annotation_names:
    raise NameError(f'"second_labels_name" {second_labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}')

if not use_custom_palette:
    color_palette = None

#Plot graph
fig = plot_embedding_interactive(df_embedding_mfasta, 
                                 labels_name = labels_name,
                                 second_labels_name = second_labels_name, 
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

In [17]:
# OPTIONS =================================================
#Here you can set different parameters to color & annotate the resulting projection:

title = "Poincaré projection of the kinase 1 protein family with embeddings"  

#----------------------------------------------------------
# Select the coloring from annotation .csv file:
labels_name = "1_Group"
# Select classes to label among the "labels_name" or "second_labels_name" column (comma separated list):
second_labels_name = ""
labels_text = []
show_text = False
#----------------------------------------------------------
# Use a custom color palette:
color_palette = kinase_palette #Default: None
use_custom_palette = True
#==========================================================


#Check projection visualization parameters
#Labels name
if labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")
#Second labels name
if second_labels_name == "":
    second_labels_name = None
elif second_labels_name not in annotation_names:
    raise NameError(f'"second_labels_name" {second_labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}')

if not use_custom_palette:
    color_palette = None

#Plot graph
fig = plot_embedding_interactive(df_embedding_embed, 
                                 labels_name = labels_name,
                                 second_labels_name = second_labels_name, 
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

In [18]:
# OPTIONS =================================================
#Here you can set different parameters to color & annotate the resulting projection:

title = "PM projection on kinases by kinase groups in UniProt without embeddings"  
#----------------------------------------------------------
# Select the coloring from annotation .csv file:
labels_name = "13_Group_in_Uni"
# Select classes to label among the "labels_name" or "second_labels_name" column (comma separated list):
second_labels_name = ""
labels_text = []
show_text = False
#----------------------------------------------------------
# Use a custom color palette:
color_palette = kinase_palette #Default: None
use_custom_palette = True
#==========================================================


#Check projection visualization parameters
#Labels name
if labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")
#Second labels name
if second_labels_name == "":
    second_labels_name = None
elif second_labels_name not in annotation_names:
    raise NameError(f'"second_labels_name" {second_labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}')

if not use_custom_palette:
    color_palette = None

#Plot graph
fig = plot_embedding_interactive(df_embedding_mfasta, 
                                 labels_name = labels_name,
                                 second_labels_name = second_labels_name, 
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

In [21]:
# OPTIONS =================================================
#Here you can set different parameters to color & annotate the resulting projection:

title = "PM projection on kinases by kinase groups in UniProt with embeddings"  

#----------------------------------------------------------
# Select the coloring from annotation .csv file:
labels_name = "13_Group_in_Uni"
# Select classes to label among the "labels_name" or "second_labels_name" column (comma separated list):
second_labels_name = ""
labels_text = []
show_text = False
#----------------------------------------------------------
# Use a custom color palette:
color_palette = kinase_palette #Default: None
use_custom_palette = True
#==========================================================


#Check projection visualization parameters
#Labels name
if labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")
#Second labels name
if second_labels_name == "":
    second_labels_name = None
elif second_labels_name not in annotation_names:
    raise NameError(f'"second_labels_name" {second_labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}')

if not use_custom_palette:
    color_palette = None

#Plot graph
fig = plot_embedding_interactive(df_embedding_embed, 
                                 labels_name = labels_name,
                                 second_labels_name = second_labels_name, 
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

### Save plot to file

In [20]:
# # OPTIONS =================================================
# output_name = "kinases_with_embeddings_by_groups"
# output_format = "html" #Format availables: ["png", "html", "pdf", "svg"]
# #==========================================================


# if output_format != "html":
#     fig.write_image(f"{output_name}.{output_format}", engine="kaleido")
# else:
#     fig.write_html(f"{output_name}.{output_format}")